# Introduction to DataFrames, Visualizations, and Statistics in Python

## [FIXME@Aditya] Add Python Introduction Here

Now run this cell to `import` the various packages that we will be using today.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import statsmodels.api as sm # FIXME: Remove in future version?

## Introduction to Pandas and Dataframes
Many of you may be familiar with the spreadsheet software Excel. In Excel, you can put anything you want into any cell you want. In data science, we work with <b>tables</b>, which are much more strictly structured. In a table, data are arranged into rows and columns such that each column is a property of whatever a row represents. You will also often hear people refer to rows of a table as "entries". 
### Pandas DataFrames
The most commonly used data management package in Python is called Pandas. What we call tables, Pandas calls <b>DataFrames</b>. You will often see DataFrames abbreviated as `df`, in code examples. Run the cell below to see a (very simple) example of a DataFrame. (Don't worry about what the cell is doing just yet.)

In [ ]:
ds_classes = pd.read_csv('./Data/01/ds_courses.csv')
ds_classes

Look at the structure of the DataFrame. Each column represents a different attribute of a row. For example, the class title corresponding to the course "Data 8" is "Foundations of Data Science". The number on the left of the DataFrame corresponds to the "index" of the row. For example, entry 4 (which is actually the 5th row from the top, because Python uses 0-based indexing!) corresponds to the row with L&S 88-2. Now that you know what a DataFrame is, let's move on to making our own!
### Making Dataframes
There are two basic ways of creating DataFrames. The first is to make one by typing the data into Python manually. For example, we can make Numpy arrays that correspond to different attributes. Note that the order of attributes in each array matters, and each array has to be the same size. Run the cell below to see an example of what this means. When we make the DataFrame itself, the input to the function, `pd.DataFrame()` is a Python dictionary with the column title and the array with the data in it.

In [ ]:
journal_titles = np.array(['Nature Reviews Molecular Cell Biology',
                           'Nature Methods',
                           'Nature Cell Biology',
                           'Cell Stem Cell',
                           'Molecular Cell',
                           'Cancer Cell',
                           'Cell Metabolism',
                           'Genome Biology',
                           'Trends in Cell Biology',
                           'Annual Review of Biophysics'])
journal_impacts = np.array([29.656, 
                            19.544, 
                            14.110, 
                            13.515, 
                            13.295, 
                            13.169, 
                            11.209, 
                            10.484, 
                            10.113, 
                            9.801])
journal_df = pd.DataFrame({'Title': journal_titles,
                           'Impact Factor': journal_impacts}) # The input to a 
journal_df

As you can see, this made us a Dataframe! However, this will obviously get very tedious for very large datasets. For large datasets formatted as CSV files, you can import the CSV file using the function, `pd.read_csv()`. (The astute reader will notice that this is the same function used above in the example). Run the following cell to see an example of this in action.

In [ ]:
family_heights = pd.read_csv('Data/01/galtonfamilies.csv')
family_heights

As you can see, this DataFrame has 934 rows and 9 columns in it! Typing this out by hand would clearly take a very very long time. 
### Working with DataFrames
Now that you know how to make DataFrames, it's time to actually do things with them! We can use `df.shape` to find the number of rows and columns in a DataFrame. The 0th entry in the output of `df.shape` is the number of rows in the DataFrame, and the 1st entry in the output of `df.shape` is the number of columsn in the DataFrame. For example, if we didn't want to scroll all the way to the bottom of the output of the previous cell, we could find the size of the `family_heights` DataFrame as follows:

In [ ]:
print('Number of rows: ' + str(family_heights.shape[0]))
print('Number of columns: ' + str(family_heights.shape[1]))

Sometimes we also might not care about some of the columns. Keeping them around can often be detrimental to the performance of Python, especially on very large datasets. Luckily, we can either choose the columns we want using `df[lst]`, where `lst` is a list of columns we want to keep, or delete the columns we don't want using `df.drop(lst, axis=1)`, where `lst` is a list of columns we don't want to keep. We need the parameter `axis=1` to tell Pandas that we are dropping a column, not a row. Here are two example cells that give us exactly the same outputs:

In [ ]:
cols_after_index = family_heights[['family', 'father', 'mother', 'gender', 'childheight']]
cols_after_index

In [ ]:
cols_after_drop = family_heights.drop(['id', 'midparentheight', 'children', 'childnum'], axis=1)
cols_after_drop

If we want to filter rows, we can simply "index" into our DataFrame using a condition. `df[df.column_name == some_value]` will give us all the rows corresponding to entries where the `column_name` property equals `some_value`. For example, let's say we only wanted to look at the heights of sons in the table of heights:

In [ ]:
only_males = family_heights[family_heights.gender == 'male']
only_males

As you can see, this returned to us a table with only the rows that had `'male'` in the `gender` column! Note that this can work with any condition. For example, if we only wanted the rows where the height was greater than a certain value, we can also index into the table with a `>` condition:

In [ ]:
taller_than_70 = family_heights[family_heights.childheight > 70]
taller_than_70

## More on DataFrames: Grouping, Pivoting, and Joining
What if we wanted to seperately study the heights of the sons and the daughters of each family? In this section we will learn how use the Pandas library to manipulate DataFrames.

Let's create a separate DataFrame containing the height information for just the parents. First, select the columns containing the family code, height of father, height of mother, and average height of parents.

In [ ]:
parent_heights = family_heights[['family','father', 'mother', 'midparentheight']]
parent_heights

You can see that each entry of the data structure still cooresponds to an individual child. We want to reduce the number of entries, so that there is only one row for every family. We can do this by **grouping** the rows of the data structure by family.

In [ ]:
parent_heights = parent_heights.groupby('family').mean()
parent_heights

Now that we have a DataFrame of `parent_heights`, let's make a DataFrame for the heights of the children! From the original DataFrame, create a new DataFrame called `children_heights` which contains the family code, gender, and height of each child.

In [ ]:
children_heights = family_heights[['family', 'gender', 'childheight']]
children_heights

Let's now look at the average daughter and son heights for each family. In this case, we can use **pivot** to find the average heights of male and female children in each of the families in the data set.

In [ ]:
average_child_height_by_gender = pd.pivot_table(children_heights, values = 'childheight', index = ['family'], columns = ['gender'])
average_child_height_by_gender

Now that we have tables for both parent and child height, we want to somehow combine the two DataFrames so we can make comparisons between parent and child height. We can use **join**, which merges two DataFrames based on a shared attribute. In this case, `parent_heights` and `average_child_height_by_gender` both have a `'family'` attribute. Joining the two tables would match columns with each other in accordance to their `'family'`.

In [ ]:
family_average_heights = parent_heights.join(average_child_height_by_gender)
family_average_heights

## Visualization: Histograms and Scatter Plots
Let's use this DataFrame to compare the children's heights to their parents! Start off with by finding the different between daughters' heights to their mothers' heights. Add these values to the `family_average_heights` DataFrame.

In [ ]:
mother_daughter_difference = family_average_heights['female'] - family_average_heights['mother']
family_average_heights['mother_daughter_height_difference'] = mother_daughter_difference

family_average_heights

Let's plot mother_daughter_height_difference on a histogram to see the distribution of height differences.

In [ ]:
family_average_heights['mother_daughter_height_difference'].plot.hist()

By looking by the histogram, we see that the **mean** of the height difference between mothers and daughters is around 0. This means, that on average, daughters tend to be around the same height as their mothers. However, the wide distribution of height differences makes it difficult for us to conclude a relationship between daughter and mother height.

Let's look at a scatterplot between mother heights and daughter heights now. **[FIXME?] Add more about how the API call works. Just showing the .scatter() function doesn't teach the students how to use it.**

In [ ]:
family_average_heights.plot.scatter(x = 'mother', y = 'female', color = 'b')

We see that there is not a clear relationship between the two heights, and are hesitant to draw any conclusions about the relationship between mother and daughter heights. **[FIXME?] Add more about why you're also plotting one for the fathers below**

In [ ]:
family_average_heights.plot.scatter(x = 'father', y = 'female', color = 'r')

Let's instead look at the relationship between the height of daughters and the average height of their parents. Create a scatter plot that compares midparentheight and female.

In [ ]:
family_average_heights.plot.scatter(x = 'midparentheight', y = 'female', color = 'g')

We can see that the average parent height and daughter height appear to have more of a linear relationship. This makes sense, since inheritance of height is dependent on both parents, not just the mother or father. 

We can further see the difference between the relationship between average parent height and daughter height and mother height and daughter height by overlaying the scatterplots that we had previously created.
**[FIXME?] Add some more notes about what the next cell is doing. What is axl? What does the ax= flag mean? etc.**

In [ ]:
axl = family_average_heights.plot.scatter(x = 'mother', y = 'female', color = 'b')
family_average_heights.plot.scatter(x = 'midparentheight', y = 'female', color = 'g', ax = axl)

## Linear Correlations and Best-Fit Lines
**[FIXME] For consistency, change this to use `family_average_heights` from above.** 
We can see that there's a positive correlation between midparent heights and child heights. In order to calculate a correlation coefficient and create a regression line, let's turn the two sets into standard units.

In [ ]:
def standard_units(numbers_array):
    return (numbers_array - np.mean(numbers_array))/np.std(numbers_array)

midparentheight_su = standard_units(family_heights['midparentheight'])
childheight_su = standard_units(family_heights['childheight'])
plt.scatter(midparentheight_su, childheight_su)

**[FIXME@Maya] Please finish the rest of the linear correlation stuff.**

## Statistics in Python
**[FIXME] Give more of an intro here (motivations, lead in from previous part, etc.)**

In [ ]:
family_heights.plot.scatter("mother", "father")

Is there a trend that you see in the scatterplot?

**[YOUR ANSWER HERE]**

Let's calculate how much taller (or shorter) each of the fathers are compared to the mothers. Extract the the heights of all the mothers and the heights of all the fathers. Subtracting a column from a column returns a column.

In [ ]:
mother_heights = family_heights["mother"]
father_heights = family_heights["father"]
difference = father_heights - mother_heights
difference

We can find the mean by calling pd.DataFrame.mean(*argument) where *argument is the column we want to find the average of. Find the mean of the differences in height.

In [ ]:
pd.DataFrame.mean(difference)

What does this tell us about the heights of a the father of a child compared to the height of the mother of a child?

**[YOUR ANSWER HERE]**

We can also visualize each of the individual height differences with a bar graph using column_name.plot.bar()

**[FIXME] This should really be a histogram. Bar graphs are used for categorical variables, while histograms are used for numerical variables. Running this cell takes a really long time because each individual height is plotted.**

In [ ]:
difference.plot.bar()

Write a short summary about the bar graph created. 

**[YOUR ANSWER HERE]**

Calculate the fraction of pairings where the mother is taller than the father. 

**[FIXME] Explain what you're doing here better.**

In [ ]:
taller_mother = sum(n<0 for n in difference)
taller_mother/len(family_heights["mother"])

To calculate mathematically whether there is indeed a correlation, we will make a linear regression model.

**[FIXME] Explain what statsmodels.api is, and what this is.**

In [ ]:
model = sm.OLS(formula='father ~ mother', data=family_heights).fit() # FIXME: This cell doesn't work on my computer.
model.summary()

$R^2$ value high? 

**[FIXME] Explain why you want to look at the R^2 value.**

**[YOUR ANSWER HERE]**